In [1]:
import pandas as pd
import numpy as np

In [113]:
import warnings
from sklearn.exceptions import UndefinedMetricWarning
from pandas.errors import SettingWithCopyWarning
warnings.filterwarnings(action='ignore', category=UndefinedMetricWarning)
warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning))
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

In [2]:
defense = pd.read_csv("../match_logs/Big5@22-23@defense.csv")
gca = pd.read_csv("../match_logs/Big5@22-23@gca.csv")
keeper = pd.read_csv("../match_logs/Big5@22-23@keeper.csv")
misc = pd.read_csv("../match_logs/Big5@22-23@misc.csv")
passing = pd.read_csv("../match_logs/Big5@22-23@passing.csv")
passing_types = pd.read_csv("../match_logs/Big5@22-23@passing_types.csv")
possession = pd.read_csv("../match_logs/Big5@22-23@possession.csv")
shooting = pd.read_csv("../match_logs/Big5@22-23@shooting.csv")

In [73]:
tactics_df = pd.read_csv("../players_db/fm23/tactics_of_teams_fc23.csv").query("fifa_version == 23 and fifa_update == 2")\
                .dropna(axis=1).drop_duplicates(subset=["team_name"], keep="first")
filtered_league_ids = [13, 16, 53, 19, 31]
tactics_df = tactics_df[tactics_df["league_id"].isin(filtered_league_ids)]

team_df = pd.read_csv("../players_db/fm23/team_ratings.csv")
triple_stats = pd.read_csv("../match_logs/Big5@22-23@triple_stats.csv")

<br><br><br><br>

In [4]:
need_cols = ["team_name", "overall", "attack", "midfield", "defence", 'def_style', 'def_team_width', 'def_team_depth', \
             'off_build_up_play', 'off_chance_creation', 'off_team_width', 'off_players_in_box', 'off_corners', 'off_free_kicks']

build_up_list = tactics_df.off_build_up_play.unique()
off_style_list = tactics_df.off_chance_creation.unique()
def_style_list = tactics_df.def_style.unique()

In [5]:
from rapidfuzz import fuzz, process
def match_names(name, choices, scorer=fuzz.ratio, threshold=70):
    best_match, score, _ = process.extractOne(name, choices, scorer=scorer)
    return best_match if score >= threshold else None

team_df['FC_Club_Name'] = team_df['Club'].apply(match_names, choices=tactics_df['team_name'])
tactics_merged_df_raw = pd.merge(team_df, 
                                 tactics_df[need_cols], 
                                 left_on="FC_Club_Name", 
                                 right_on="team_name", 
                                 how="inner")
tactics_merged_df = tactics_merged_df_raw.copy()
print(tactics_merged_df_raw.shape)
# stats_df[["FC_Club_Name","team_name","Club"]].tail(5)

(91, 59)


In [6]:
stats_df = pd.merge(tactics_merged_df, triple_stats, left_on="Club", right_on="Club", how="inner")

In [7]:
stats_df[["overall","tpr"]].mean()

overall    76.505495
tpr        71.010989
dtype: float64

In [8]:
stats_df.columns

Index(['Club', 'GK', 'Based', 'Division', 'Club_id', 'League_id', 'tpr',
       'Anticipation', 'Stamina', 'Balance', 'Bravery', 'Marking', 'Pace',
       'Off_the_Ball', 'Heading', 'Crossing', 'Vision', 'Tackling',
       'Decisions', 'Natural_Fitness', 'Positioning', 'Jumping_Reach',
       'Acceleration', 'Dribbling', 'Finishing', 'Flair', 'Free_Kick_Taking',
       'Agility', 'Penalty_Taking', 'Aggression', 'Long_Throws',
       'Concentration', 'Composure', 'Technique', 'Corners', 'Leadership',
       'Teamwork', 'Strength', 'Determination', 'Work_Rate', 'Long_Shots',
       'Passing', 'First_Touch', 'fbref_name', 'FC_Club_Name', 'team_name',
       'overall', 'attack', 'midfield', 'defence', 'def_style',
       'def_team_width', 'def_team_depth', 'off_build_up_play',
       'off_chance_creation', 'off_team_width', 'off_players_in_box',
       'off_corners', 'off_free_kicks', 'Att_pass', 'Att_shot', 'Att_dribble',
       'Total_touches', '%pass', '%shot', '%dribble'],
      dtype=

In [9]:
# grp_cols = ["overall","tpr","Total_touches","Att_pass","Att_shot","Att_dribble","%pass","%shot","%dribble"]
grp_cols = ["tpr","Total_touches","%pass","%shot","%dribble"]
query = "tpr < 70"
stats_df.query(query).shape[0]

50

In [10]:
stats_df.query(query).groupby(by="off_build_up_play").mean(numeric_only=True)[grp_cols].round(3)

,tpr,Total_touches,%pass,%shot,%dribble
off_build_up_play,,,,,
Balanced,65.750,478.341,0.937,0.024,0.040
Fast Build Up,66.071,479.133,0.940,0.023,0.036
Long Ball,66.857,432.648,0.937,0.026,0.037
Slow Build Up,66.471,492.960,0.939,0.024,0.037


In [11]:
stats_df.query(query).groupby(by="off_chance_creation").mean(numeric_only=True)[grp_cols].round(3)

,tpr,Total_touches,%pass,%shot,%dribble
off_chance_creation,,,,,
Balanced,65.750,478.341,0.937,0.024,0.040
Direct Passing,66.857,432.648,0.937,0.026,0.037
Forward Runs,66.071,479.133,0.940,0.023,0.036
Possession,66.471,492.960,0.939,0.024,0.037


In [12]:
stats_df.query(query).groupby(by="def_style").mean(numeric_only=True)[grp_cols].round(3)

,tpr,Total_touches,%pass,%shot,%dribble
def_style,,,,,
Balanced,66.423,477.263,0.939,0.024,0.038
Press After Possession Loss,67.200,470.425,0.938,0.024,0.037
Pressure On Heavy Touch,65.214,481.695,0.939,0.024,0.037


In [75]:
column_name = "tpr"
interval_size = 5
min_value = stats_df[column_name].min()
max_value = stats_df[column_name].max()

def group_interval(value, min_val, interval):
    return int((value - min_val) // interval) * interval + min_val

stats_df["group_interval"] = stats_df[column_name].apply(lambda x: group_interval(x, min_value, interval_size))
grouped_df = stats_df.groupby("group_interval").mean(numeric_only=True)[grp_cols]

In [153]:
matches_df_raw_1 = pd.merge(tactics_merged_df, gca, left_on="Club", right_on="Club", how="inner")
matches_df_raw_2 = pd.merge(tactics_merged_df, matches_df_raw_1, left_on="fbref_name", right_on="Opponent", how="inner", suffixes=("_x", "_y"))
matches_df_raw_3 = pd.merge(triple_stats, matches_df_raw_2, left_on="Club", right_on="Club_x", how="inner")
matches_df_raw_4 = pd.merge(triple_stats, matches_df_raw_3, left_on="Club", right_on="Club_y", how="inner", suffixes=("_x", "_y"))
matches_df = matches_df_raw_4.copy()
matches_df["GF"] = matches_df["GF"].astype(int)
matches_df["GA"] = matches_df["GA"].astype(int)

In [164]:
m_cols = ["Club_x", "GF", "GA", "Club_y", "Result", "tpr_x", "tpr_y", "off_build_up_play_x", "off_chance_creation_x", "off_team_width_x",
          "def_style_y", 'def_team_width_y', 'def_team_depth_y']
off_def_cols = ["off_build_up_play_x", "off_chance_creation_x", "off_team_width_x",
               "def_style_y", 'def_team_width_y', 'def_team_depth_y']
for col in off_def_cols:
    matches_df[f'{col}_Code'] = pd.Categorical(matches_df[col]).codes
off_def_codes = matches_df.columns[matches_df.columns.str.contains("_Code")]

In [181]:
matches_df['Strategy'] = matches_df.apply(
    lambda row: f"{row['off_build_up_play_x']} {row['off_chance_creation_x']} {row['off_team_width_x']}"
                f" DEF: {row['def_style_y']} {row['def_team_width_y']} {row['def_team_depth_y']}",
    axis=1
)

matches_df['Strategy'] = matches_df.apply(
    lambda row: f"{row['off_build_up_play_x']} {row['off_chance_creation_x']}"
                f"--- DEF: {row['def_style_y']} {row['def_team_width_y']} {row['def_team_depth_y']}",
    axis=1
)
# matches_df.groupby("Strategy").mean()[["Att_pass_x","Att_shot_x","Att_dribble_x"]]

In [183]:
grouped = matches_df.groupby(
    # ["off_build_up_play_x", "off_chance_creation_x", "def_style_y", 'def_team_width_y', "Result"]
     #    ["off_build_up_play_x", "off_chance_creation_x", "off_team_width_x",
     # "def_style_y", 'def_team_width_y', 'def_team_depth_y',
     # "Result"]
    ["Strategy", "Result"]
).size().unstack(fill_value=0)

total_matches = grouped.sum(axis=1)
win_rate = grouped['W'] / total_matches
loss_rate = grouped['L'] / total_matches
draw_rate = grouped['D'] / total_matches

rates_df = pd.DataFrame({
    'Win %': win_rate,
    'Loss %': loss_rate,
    'Draw %': draw_rate,
    'N': total_matches,
}).fillna(0)

# rates_df.to_excel("./data/rates_df_1.xlsx", index=False)
rates_df.sort_values("Win %", ascending=False).head(20)

,Win %,Loss %,Draw %,N
Strategy,,,,
Long Ball Direct Passing--- DEF: Balanced 51.0 71.0,1.000000,0.000000,0.000000,4
Long Ball Direct Passing--- DEF: Pressure On Heavy Touch 61.0 81.0,1.000000,0.000000,0.000000,4
Slow Build Up Possession--- DEF: Press After Possession Loss 51.0 81.0,0.875000,0.125000,0.000000,8
Balanced Balanced--- DEF: Pressure On Heavy Touch 61.0 81.0,0.833333,0.166667,0.000000,12
Fast Build Up Forward Runs--- DEF: Press After Possession Loss 51.0 81.0,0.800000,0.200000,0.000000,10
Long Ball Direct Passing--- DEF: Balanced 71.0 71.0,0.750000,0.000000,0.250000,4
Long Ball Direct Passing--- DEF: Press After Possession Loss 71.0 71.0,0.750000,0.000000,0.250000,4
Fast Build Up Forward Runs--- DEF: Press After Possession Loss 71.0 81.0,0.750000,0.250000,0.000000,8
Long Ball Direct Passing--- DEF: Pressure On Heavy Touch 71.0 61.0,0.750000,0.250000,0.000000,4
